Actividad 4: Deteccion de fraudes a partir de autoencoders

La deteccion de fraudes es un problema de actualidad y que se ha venido trabajando con ayuda de deeplearning. En este caso realizaremos el ejercicio de entrenar un modelo de autoencoder para tratar de detectar fraudes.


In [ ]:
!git clone https://github.com/eyberthrojas/credit_cart.git

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
pd.options.display.max_columns = 100

## Extrayendo los datos

In [4]:
data = pd.read_parquet('credit_cart/creditcard.parquet')

In [ ]:
data.shape

In [ ]:
data.sample(5, random_state=0)

Se cuenta con un dataset de 60492 registros, tenemos 29 variables de entrada, columnas V1-V28 mas columna Amount, y la etiqueta que en este caso se denomina Class

# Distribucion de los labels:

In [ ]:
data['Class'].value_counts()

La idea del modelo, es entrenar el encoder con solo datos con transacciones normales, por lo tanto el encoder entregara valores muy parecidos  a los de entrada en el caso que sean de transacciones normales, en el caso que ingrese un dato fraudulenteo, el encoder no conoce este tipo de informacion y va a entregar informacion muy diferente a la de entrada.

In [8]:
# Primero definamos datos de train y test

df_normal = data[data['Class'] == 0].copy()
df_fraude = data[data['Class'] == 1].copy()

In [9]:
df_train, df_test_normal = train_test_split(df_normal, test_size=0.2, random_state=42)
df_train = df_train.drop(['Class'], axis=1)

In [10]:
df_test = pd.concat([df_test_normal, df_fraude]).sample(frac=1, random_state=0)

In [ ]:
df_test['Class'].value_counts()

In [12]:
X_train = df_train.to_numpy()
y_test = df_test['Class'].to_numpy()
X_test = df_test.drop(['Class'], axis=1).to_numpy()

# Construccion del autoencoder
Sabemos que tenemos 29 caracateristicas, por lo que la salida del decoder es tambien de 29 caracteristicas.

# Definicion de hyperparametros

In [13]:
# Número de caracteristicas
n_features = df_train.shape[1]
# Numero de neuronas por capas
neurons = [20, 8, 20, 29]
# unciones de activación para la respectiva capa
activations = ['tanh', 'relu', 'tanh', 'relu']
# learning rate
learning_rate=0.001
# Número de épocas
epochs=100

In [14]:
model = Sequential()
model.add(InputLayer((n_features,)))
for neuron, activation in zip(neurons, activations):
  model.add(Dense(neuron, activation=activation))

In [15]:
# Función de costo 
loss = tf.keras.losses.MeanSquaredError()
# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss=loss)

In [ ]:
model.summary()

In [ ]:
history = model.fit(x=X_train, y=X_train, batch_size=32, epochs=epochs, verbose=True)

## Validacion del modelo
Como el modelo fue entrenado sólo con transacciones normales, es de esperar que al introducir un registro fraudulento la reconstrucción del dato no sea tan precisa y por tanto la diferencia entre el dato reconstruido y el original será más grande que la obtenida cuando el dato ingresado es normal

Así que para realizar la detección de fraudes seguiremos este procedimiento:

- En primer lugar tomaremos el Autoencoder entrenado y lo usaremos para generar una predicción, usando el set de validación (X_test)
- Luego calcularemos el error cuadrático medio entre el dato original (a la entrada del Autoencoder) y el dato reconstruido (generado a través de la predicción)
- Posteriormente estableceremos un umbral: si el error calculado en el punto anterior supera el umbral, tendremos un registro “fraudulento”, y en caso contrario tendremos un registro “normal”.

In [ ]:
X_pred = model.predict(X_test)

In [18]:
ecm = np.mean(np.power(X_test-X_pred,2), axis=1)

In [23]:
umbral_fijo = 0.99
Y_pred = [1 if e > umbral_fijo else 0 for e in ecm]

In [ ]:
print(classification_report(y_test, Y_pred, zero_division=0))